In [12]:
import numpy as np
import cv2
import os
from skimage.restoration import denoise_nl_means, estimate_sigma
from sklearn.model_selection import train_test_split

In [13]:
def leer_imagenes(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

In [14]:
def convertir_a_grises(images):
    gray_images = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in images]
    gray_images = [np.expand_dims(img, axis=-1) for img in gray_images]  # Aseguramos que las imágenes tengan el canal
    return gray_images


In [15]:
from skimage.restoration import denoise_nl_means, estimate_sigma

def aplicar_filtro_anisotropico(images):
    filtered_images = []
    for image in images:
        # Asegúrate de que image tiene la forma esperada, o ajusta el channel_axis
        # Si las imágenes son (altura, anchura, 1), usa channel_axis=-1
        sigma_est = np.mean(estimate_sigma(image, channel_axis=-1))
        denoised = denoise_nl_means(image, h=1.15 * sigma_est, fast_mode=True,
                                    patch_size=5, patch_distance=6)
        filtered_images.append(denoised)
    return filtered_images


In [16]:
def generar_muestras(images, filtered_images, k=32):
    x_list = []
    y_list = []
    for img, filtered_img in zip(images, filtered_images):
        x, y = np.random.randint(0, img.shape[0] - k), np.random.randint(0, img.shape[1] - k)
        original_patch = img[x:x+k, y:y+k]
        filtered_patch = filtered_img[x:x+k, y:y+k]
        x_list.append(original_patch)
        y_list.append(filtered_patch)
    return np.array(x_list), np.array(y_list)  


In [17]:
def main(folder='test', k=32):
    images = leer_imagenes(folder)
    gray_images = convertir_a_grises(images)
    filtered_images = aplicar_filtro_anisotropico(gray_images)
    x, y = generar_muestras(gray_images, filtered_images, k=k)
    return x, y

# Ejecutar la función main
x, y = main()
X_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [ ]:
input_shape = (32, 32, 1)
unet = UNet()
model = unet.build_model(input_shape)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
    x=X_train, 
    y=y_train, 
    epochs=20, 
    validation_data=(x_test, y_test)
)